# Initialize Environment

In [ ]:
!pip install -U /kaggle/input/kerasapplications

In [ ]:
!pip install -U /kaggle/input/efficientnet/efficientnet-master

In [ ]:
!pip install -U /kaggle/input/tensorflowresnets/TensorFlow-ResNets

In [ ]:
def is_interactive():
    return 'runtime'    in get_ipython().config.IPKernelApp.connection_file
IS_INTERACTIVE = is_interactive()
print(IS_INTERACTIVE)

In [ ]:
import os 
IS_PRIVATE = len(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))!=1
print(IS_PRIVATE)

In [ ]:
import pandas as pd, numpy as np
from kaggle_datasets import KaggleDatasets
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import efficientnet.tfkeras as efn
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report, precision_recall_curve
from IPython.display import display
import gc
import cv2

from tf2_resnets import models 

# TFRecords Creation

In [ ]:
def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image,image_name):
    feature = {
        'image': _bytes_feature(image),
        'image_name': _bytes_feature(image_name),
      }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [ ]:
import os

RESNEXT_ID = 10
N_TFRECORDS = 20
IMAGE_HEIGHT = 600
IMAGE_WIDTH = 800
os.mkdir('test_tfrecords_600')
test_df = pd.DataFrame(os.listdir('../input/cassava-leaf-disease-classification/test_images/'),
                      columns=['image_name'])
test_df['tfr_group'] = test_df.index%N_TFRECORDS

In [ ]:
for tfr_group in range(N_TFRECORDS):
    df = test_df[test_df.tfr_group==tfr_group]
    if df.shape[0]>0:
        tfr_filename = 'test_tfrecords_600/cassava_test{}-{}.tfrec'.format(tfr_group,df.shape[0])
        print("Writing",tfr_filename)
        with tf.io.TFRecordWriter(tfr_filename) as writer:
            for index,row in tqdm(df.iterrows()):
                image_name = row['image_name']
                image_path = '../input/cassava-leaf-disease-classification/test_images/'+image_name
                image = cv2.imread(image_path)
                image = cv2.resize(image,(IMAGE_WIDTH,IMAGE_HEIGHT))           
                image_shape = image.shape
                #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Fix incorrect colors
                image = cv2.imencode('.jpg', image, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
                image_name = str.encode(image_name)
                sample = serialize_example(image,image_name)
                writer.write(sample)
image_shape

In [ ]:
import os
N_TFRECORDS = 20
IMAGE_HEIGHT = 666
IMAGE_WIDTH = 500
os.mkdir('test_tfrecords_500')
test_df = pd.DataFrame(os.listdir('../input/cassava-leaf-disease-classification/test_images/'),
                      columns=['image_name'])
test_df['tfr_group'] = test_df.index%N_TFRECORDS

In [ ]:
for tfr_group in range(N_TFRECORDS):
    df = test_df[test_df.tfr_group==tfr_group]
    if df.shape[0]>0:
        tfr_filename = 'test_tfrecords_500/cassava_test{}-{}.tfrec'.format(tfr_group,df.shape[0])
        print("Writing",tfr_filename)
        with tf.io.TFRecordWriter(tfr_filename) as writer:
            for index,row in tqdm(df.iterrows()):
                image_name = row['image_name']
                image_path = '../input/cassava-leaf-disease-classification/test_images/'+image_name
                image = cv2.imread(image_path)
                image = cv2.resize(image,(IMAGE_WIDTH,IMAGE_HEIGHT))           
                image_shape = image.shape
                #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Fix incorrect colors
                image = cv2.imencode('.jpg', image, (cv2.IMWRITE_JPEG_QUALITY, 100))[1].tostring()
                image_name = str.encode(image_name)
                sample = serialize_example(image,image_name)
                writer.write(sample)
image_shape

## Configuration
In order to be a proper cross validation with a meaningful overall CV score (aligned with LB score), **you need to choose the same** `IMG_SIZES`, `INC2019`, `INC2018`, and `EFF_NETS` **for each fold**. If your goal is to just run lots of experiments, then you can choose to have a different experiment in each fold. Then each fold is like a holdout validation experiment. When you find a configuration you like, you can use that configuration for all folds. 
* DEVICE - is GPU or TPU
* SEED - a different seed produces a different triple stratified kfold split.
* FOLDS - number of folds. Best set to 3, 5, or 15 but can be any number between 2 and 15
* IMG_SIZES - is a Python list of length FOLDS. These are the image sizes to use each fold
* INC2019 - This includes the new half of the 2019 competition data. The second half of the 2019 data is the comp data from 2018 plus 2017
* INC2018 - This includes the second half of the 2019 competition data which is the comp data from 2018 plus 2017
* BATCH_SIZES - is a list of length FOLDS. These are batch sizes for each fold. For maximum speed, it is best to use the largest batch size your GPU or TPU allows.
* EPOCHS - is a list of length FOLDS. These are maximum epochs. Note that each fold, the best epoch model is saved and used. So if epochs is too large, it won't matter.
* EFF_NETS - is a list of length FOLDS. These are the EfficientNets to use each fold. The number refers to the B. So a number of `0` refers to EfficientNetB0, and `1` refers to EfficientNetB1, etc.
* WGTS - this should be `1/FOLDS` for each fold. This is the weight when ensembling the folds to predict the test set. If you want a weird ensemble, you can use different weights.
* TTA - test time augmentation. Each test image is randomly augmented and predicted TTA times and the average prediction is used. TTA is also applied to OOF during validation.

In [ ]:
DEVICE = "GPU" #or "GPU"

# NUMBER OF FOLDS. USE 3, 5, OR 15 
FOLDS = 5

FOLD_TO_RUN = [0,1,2,3,4]

# WHICH IMAGE SIZES TO LOAD EACH FOLD
# CHOOSE 128, 192, 256, 384, 512, 768 


# BATCH SIZE AND EPOCHS
BATCH_SIZE = 128
EPOCHS = 15

N_WORKERS = 4


# TEST TIME AUGMENTATION STEPS



In [ ]:
if DEVICE == "TPU":
    print("connecting to TPU...")
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        print("Could not connect to TPU")
        tpu = None

    if tpu:
        try:
            print("initializing  TPU ...")
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.experimental.TPUStrategy(tpu)
            print("TPU initialized")
        except _:
            print("failed to initialize TPU")
    else:
        DEVICE = "GPU"

if DEVICE != "TPU":
    print("Using default strategy for CPU and single GPU")
    strategy = tf.distribute.get_strategy()

if DEVICE == "GPU":
    print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
    

AUTO     = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

# Step 1: Preprocess
Preprocess has already been done and saved to TFRecords. Here we choose which size to load. We can use either 128x128, 192x192, 256x256, 384x384, 512x512, 768x768 by changing the `IMG_SIZES` variable in the preceeding code section. These TFRecords are discussed [here][1]. The advantage of using different input sizes is discussed [here][2]

[1]: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/155579
[2]: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/160147

In [ ]:
GCS_PATH = '.'
files_test_600 = np.sort(np.array(tf.io.gfile.glob(GCS_PATH + '/test_tfrecords_600/*.tfrec')))
files_test_500 = np.sort(np.array(tf.io.gfile.glob(GCS_PATH + '/test_tfrecords_500/*.tfrec')))
print(files_test_600)
print(files_test_500)

# Step 2: Data Augmentation
This notebook uses rotation, sheer, zoom, shift augmentation first shown in this notebook [here][1] and successfully used in Melanoma comp by AgentAuers [here][2]. This notebook also uses horizontal flip, hue, saturation, contrast, brightness augmentation similar to last years winner and also similar to AgentAuers' notebook.

Additionally we can decide to use external data by changing the variables `INC2019` and `INC2018` in the preceeding code section. These variables respectively indicate whether to load last year 2019 data and/or year 2018 + 2017 data. These datasets are discussed [here][3]

Consider experimenting with different augmenation and/or external data. The code to load TFRecords is taken from AgentAuers' notebook [here][2]. Thank you AgentAuers, this is great work.

[1]: https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
[2]: https://www.kaggle.com/agentauers/incredible-tpus-finetune-effnetb0-b6-at-once
[3]: https://www.kaggle.com/c/siim-isic-melanoma-classification/discussion/164910

In [ ]:
ROT_ = 180.0
SHR_ = 2.0
HZOOM_ = 8.0
WZOOM_ = 8.0
HSHIFT_ = 8.0
WSHIFT_ = 8.0

In [ ]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear    = math.pi * shear    / 180.

    def get_3x3_mat(lst):
        return tf.reshape(tf.concat([lst],axis=0), [3,3])
    
    # ROTATION MATRIX
    c1   = tf.math.cos(rotation)
    s1   = tf.math.sin(rotation)
    one  = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    
    rotation_matrix = get_3x3_mat([c1,   s1,   zero, 
                                   -s1,  c1,   zero, 
                                   zero, zero, one])    
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)    
    
    shear_matrix = get_3x3_mat([one,  s2,   zero, 
                                zero, c2,   zero, 
                                zero, zero, one])        
    # ZOOM MATRIX
    zoom_matrix = get_3x3_mat([one/height_zoom, zero,           zero, 
                               zero,            one/width_zoom, zero, 
                               zero,            zero,           one])    
    # SHIFT MATRIX
    shift_matrix = get_3x3_mat([one,  zero, height_shift, 
                                zero, one,  width_shift, 
                                zero, zero, one])
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), 
                 K.dot(zoom_matrix,     shift_matrix))


def transform(image, DIM=256):    
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    XDIM = DIM%2 #fix for size 331
    
    rot = ROT_ * tf.random.normal([1], dtype='float32')
    shr = SHR_ * tf.random.normal([1], dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32') / HZOOM_
    w_zoom = 1.0 + tf.random.normal([1], dtype='float32') / WZOOM_
    h_shift = HSHIFT_ * tf.random.normal([1], dtype='float32') 
    w_shift = WSHIFT_ * tf.random.normal([1], dtype='float32') 

    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x   = tf.repeat(tf.range(DIM//2, -DIM//2,-1), DIM)
    y   = tf.tile(tf.range(-DIM//2, DIM//2), [DIM])
    z   = tf.ones([DIM*DIM], dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m, tf.cast(idx, dtype='float32'))
    idx2 = K.cast(idx2, dtype='int32')
    idx2 = K.clip(idx2, -DIM//2+XDIM+1, DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack([DIM//2-idx2[0,], DIM//2-1+idx2[1,]])
    d    = tf.gather_nd(image, tf.transpose(idx3))
        
    return tf.reshape(d,[DIM, DIM,3])

In [ ]:
def read_unlabeled_tfrecord(example):
    tfrec_format = {
        'image'                        : tf.io.FixedLenFeature([], tf.string),
        "image_name": tf.io.FixedLenFeature([], tf.string)
    }           
    example = tf.io.parse_single_example(example, tfrec_format)
    return example['image'], example['image_name']
 
def prepare_image(img, augment=True, tta=None, dim=256):    
    img = tf.image.decode_jpeg(img, channels=3)
    
    if dim==600:
        img = tf.image.crop_to_bounding_box(img, 0, 100, 600, 600)
    else:
        img = tf.image.crop_to_bounding_box(img, 83, 0, 500, 500)
        
    img = tf.image.resize(img, [dim,dim])
    
    img = tf.cast(img, tf.float32) / 255.0
    
    if tta!=None:
        if tta[-3:]=='_lr':
            img = tf.image.flip_left_right(img)
            tta = tta[:-3]
            
        if tta=='rotate90':
            img = tf.image.rot90(img,1)
            
        if tta=='rotate180':
            img = tf.image.rot90(img,2)
            
        if tta=='rotate270':
            img = tf.image.rot90(img,3)
                      
    img = tf.reshape(img, [dim,dim, 3])
            
    return img

def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) 
         for filename in filenames]
    return np.sum(n)
count_data_items(files_test_600),count_data_items(files_test_500)

In [ ]:
def get_dataset(files, augment = False, tta=None, shuffle = False, repeat = False, 
                batch_size=16, dim=512):
    
    ds = tf.data.TFRecordDataset(files, num_parallel_reads=AUTO)
    
    if repeat:
        ds = ds.repeat()
    
    if shuffle: 
        ds = ds.shuffle(1024*8)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        ds = ds.with_options(opt)
    
    ds = ds.map(read_unlabeled_tfrecord, num_parallel_calls=AUTO)
   
    
    ds = ds.map(lambda img, imgname_or_label: (prepare_image(img, augment=augment, tta=tta, dim=dim), 
                                               imgname_or_label), 
                num_parallel_calls=AUTO)
    
    ds = ds.batch(batch_size * REPLICAS)
    ds = ds.prefetch(AUTO)
    return ds

# Step 3: Build Model
This is a common model architecute. Consider experimenting with different backbones, custom heads, losses, and optimizers. Also consider inputing meta features into your CNN.

In [ ]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, efn.EfficientNetB6, efn.EfficientNetB7]

def build_model(dim=128, ef=0):
    inp = tf.keras.layers.Input(shape=(dim,dim,3))
    if ef == RESNEXT_ID:
        base = models.ResNeXt50(input_shape=(dim,dim,3),weights=None,include_top=False)
    else:
        base = EFNS[ef](input_shape=(dim,dim,3),weights=None,include_top=False)
    x = base(inp)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(128,activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(5,activation='softmax')(x)
    model = tf.keras.Model(inputs=inp,outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    model.compile(optimizer=opt,loss=loss,metrics=[metrics])
    return model
build_model().summary()

In [ ]:
def get_lr_callback(batch_size=8):
    lr_start   = 0.000005
    lr_max     = 0.00000125 * REPLICAS * batch_size
    lr_min     = 0.000001
    lr_ramp_ep = 5
    lr_sus_ep  = 0
    lr_decay   = 0.8
   
    def lrfn(epoch):
        if epoch < lr_ramp_ep:
            lr = (lr_max - lr_start) / lr_ramp_ep * epoch + lr_start
            
        elif epoch < lr_ramp_ep + lr_sus_ep:
            lr = lr_max
            
        else:
            lr = (lr_max - lr_min) * lr_decay**(epoch - lr_ramp_ep - lr_sus_ep) + lr_min
            
        return lr

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=False)
    return lr_callback

## Get Test IDs

In [ ]:
print('Getting test_ids')
IMG_SIZE = 500
NUM_TEST_IMAGES = count_data_items(files_test_500)
ds_test = get_dataset(files_test_500,augment=False,repeat=False,shuffle=False,
                                   dim=IMG_SIZE,batch_size=BATCH_SIZE*4)
test_ids_ds = ds_test.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch


In [ ]:
test_ids.shape

## INFERENCE

In [ ]:
tta_counter = 0

In [ ]:
# USE VERBOSE=0 for silent, VERBOSE=1 for interactive, VERBOSE=2 for commit
VERBOSE = IS_INTERACTIVE 
def generate_submission(EFF_NET,category,TTA):
    
    if EFF_NET == RESNEXT_ID:
        model_root = f'../input/cassava-category-{category}/ResNext50/ResNext50/'
    else:
        model_root = f'../input/cassava-category-{category}/B{EFF_NET}/B{EFF_NET}/'
    
    if category%2==0:
        IMG_SIZE = 600
        files_test = files_test_600
    elif category%2==1:
        IMG_SIZE = 500
        files_test = files_test_500
    else:
        assert 1==2, "INVALID CATEGORY"
           
    global tta_counter
    test_predictions = []
    for fold in range(FOLDS):

        # DISPLAY FOLD INFO
        print('#'*25); print('#### FOLD',fold+1)
        if EFF_NET == RESNEXT_ID:
            print('#### Image Size %i with ResNext50 and batch_size %i'%
                  (IMG_SIZE,BATCH_SIZE*REPLICAS))
        else:
            print('#### Image Size %i with EfficientNet B%i and batch_size %i'%
                  (IMG_SIZE,EFF_NET,BATCH_SIZE*REPLICAS))


        # BUILD MODEL
        K.clear_session()
        with strategy.scope():
            model = build_model(dim=IMG_SIZE,ef=EFF_NET)

        print('Loading best model...')
        model.load_weights(model_root+'fold-%i.h5'%fold)

        if len(TTA)==0:
            print('Predicting TEST without TTA...')
            ds_test = get_dataset(files_test,augment=False,repeat=False,shuffle=False,
                                   dim=IMG_SIZE,batch_size=BATCH_SIZE)
            pred = model.predict(ds_test,verbose=VERBOSE,batch_size=BATCH_SIZE,
                                 use_multiprocessing=True,workers=N_WORKERS)

        if 1:
            print('Predicting TEST with TTA...')
            for x_ in range(1):
                tta = TTA[tta_counter%8]
                print(tta)
                tta_counter+=1
                ds_test = get_dataset(files_test,augment=False,tta=tta,repeat=False,shuffle=False,
                                       dim=IMG_SIZE,batch_size=BATCH_SIZE)
                if x_==0:
                    pred = model.predict(ds_test,verbose=VERBOSE,batch_size=BATCH_SIZE,
                                         use_multiprocessing=True,workers=N_WORKERS)/len(TTA)
                else:
                    pred += model.predict(ds_test,verbose=VERBOSE,batch_size=BATCH_SIZE,
                                          use_multiprocessing=True,workers=N_WORKERS)/len(TTA)

        test_pred_fold = pd.DataFrame(pred)
        test_pred_fold['fold'] = fold
        test_pred_fold['image_id'] = test_ids
        test_predictions.append(test_pred_fold)
    test_predictions = pd.concat(test_predictions)
    test_predictions_averaged = test_predictions.groupby('image_id')[[0,1,2,3,4]].mean()
    test_predictions_averaged = test_predictions_averaged.sort_index()
    return test_predictions_averaged

In [ ]:
CASSAVA_WEIGHTS = {(1, 'B0'): 0.0,
 (1, 'B1'): 0.0,
 (3, 'ResNext50'): 0.2144082584031884,
 (4, 'B0'): 0.04051596743735366,
 (4, 'B1'): 0.0059384033956569274,
 (4, 'B2'): 0.0,
 (4, 'B3'): 0.2772041683999385,
 (4, 'B4'): 1.0,
 (4, 'B5'): 1.0,
 (4, 'ResNext50'): 1.0,
 (5, 'B3'): 0.47634532056167095,
 (5, 'B4'): 0.0,
 (1, 'B2'): 0.0,
 (6, 'B4'): 0.81263538458694,
 (7, 'B1'): 0.6091246357587939,
 (7, 'B2'): 0.8727553512599798,
 (7, 'B3'): 1.0,
 (7, 'B4'): 0.0,
 (7, 'ResNext50'): 0.0,
 (8, 'B3'): 0.36729055080178946,
 (8, 'B4'): 0.3756019549511829,
 (8, 'ResNext50'): 1.0,
 (1, 'B3'): 0.0,
 (1, 'B4'): 1.0,
 (3, 'B0'): 0.0,
 (3, 'B1'): 0.0,
 (3, 'B2'): 0.0,
 (3, 'B3'): 0.3450946319830929,
 (3, 'B4'): 0.0}

In [ ]:
submission = pd.DataFrame(test_ids,columns=['image_id'])
for x in range(5):
    submission[x] = 0
submission = submission.set_index('image_id')
submission = submission.sort_index()
submission

In [ ]:
import random

In [ ]:
TTA = ['rotate0', 'rotate90', 'rotate180','rotate270', 'rotate0_lr', 'rotate90_lr', 'rotate180_lr','rotate270_lr']
for category,effnet in CASSAVA_WEIGHTS:
    random.shuffle(TTA)
    wt = CASSAVA_WEIGHTS[(category,effnet)]
    if effnet == 'ResNext50':
        EFF_NET = RESNEXT_ID
    else:
        EFF_NET = int(effnet[1:])
    if wt>0.1:
        print(category,effnet,wt)
        try:
            sub = generate_submission(EFF_NET,category,TTA)
            submission += sub*wt
        except:
            print("Exception Raised")
submission

# Submission

In [ ]:
submission.to_csv('predictions.csv')
submission

In [ ]:
!rm -r test_tfrecords_600/

In [ ]:
!rm -r test_tfrecords_500/

In [ ]:
os.listdir('.')

In [ ]:
K.clear_session()

## VIT


https://www.kaggle.com/zekun98/xla-vision-transformer-vit?scriptVersionId=53884787 (V19)

In [ ]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

In [ ]:
import sys

package_paths = [
    '../input/pytorch-image-models/pytorch-image-models-master', #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
    '../input/image-fmix/FMix-master'
]
for pth in package_paths:
    sys.path.append(pth)
import timm

In [ ]:
from glob import glob
from sklearn.model_selection import GroupKFold, StratifiedKFold
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import cv2
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from  torch.cuda.amp import autocast, GradScaler

import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
import cv2
import pydicom
import timm #from efficientnet_pytorch import EfficientNet
from scipy.ndimage.interpolation import zoom
from sklearn.metrics import log_loss

In [ ]:
# Import libraries
import os
import pandas as pd
import albumentations as albu
import matplotlib.pyplot as plt
import json
import seaborn as sns
import cv2
import albumentations as albu
import numpy as np

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold, train_test_split
from albumentations.pytorch import ToTensorV2
# from efficientnet_pytorch import EfficientNet
import time
import datetime
import copy

In [ ]:
CFG = {
    'fold_num': 10,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b3_ns',
    'img_size': 384,
    'epochs': 32,
    'train_bs': 32,
    'valid_bs': 32,
    'lr': 1e-4,
    'num_workers': 4,
    'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
    'verbose_step': 1,
    'device': 'cuda:0',
    'tta': 4,
    'used_epochs': [8],#[6,7,8,9],
    'weights': [1,1,1,1,1]
}

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

In [ ]:
# DataSet class

class CassavaDataset(Dataset):
    def __init__(self,df:pd.DataFrame,imfolder:str,train:bool = True, transforms=None):
        self.df=df
        self.imfolder=imfolder
        self.train=train
        self.transforms=transforms
        
    def __getitem__(self,index):
        im_path=os.path.join(self.imfolder,self.df.iloc[index]['image_id'])
        x=cv2.imread(im_path,cv2.IMREAD_COLOR)
        x=cv2.cvtColor(x,cv2.COLOR_BGR2RGB)
        
        if(self.transforms):
            x=self.transforms(image=x)['image']
        
        if(self.train):
            y=self.df.iloc[index]['label']
            return x,y
        else:
            return x
        
    def __len__(self):
        return len(self.df)

In [ ]:

# model_name = 'efficientnet-b7'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class CustomDeiT(nn.Module):
    def __init__(self, model_name='model_name', pretrained=False):
        super().__init__()
        self.model = torch.hub.load('facebookresearch/deit:main', model_name, pretrained=0)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, CFG.target_size)

    def forward(self, x):
        x = self.model(x)
        return x
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        print(self.model)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features, n_class)

    def forward(self, x):
        x = self.model(x)
        return x
    
class CustomViT(nn.Module):
    def __init__(self, model_name='', pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.head.in_features
        self.model.head = nn.Linear(n_features,5)

    def forward(self, x):
        x = self.model(x)
        return x
# model = create_model(
#         'deit_base_patch16_224',
#         pretrained=False,
#         num_classes=5,
        
#         drop_block_rate=None,
#     ).to(device)
# model=models.(pretrained=True)
# model.fc=nn.Linear(512,5)
# model = EfficientNet.from_pretrained(model_name, num_classes=5) 
# model=models.resnext50_32x4d()#Add Pretrained=True to use pretrained with internet enabled
# model.fc=nn.Linear(model.fc.in_features,5)
model = CustomViT(model_name='vit_base_patch16_384', pretrained=False).to(device)
device = torch.device(CFG['device'])
model.to(device)
model.eval()

In [ ]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['label']
          
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [ ]:
from albumentations.pytorch import ToTensorV2

from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout,RandomCrop, ShiftScaleRotate, CenterCrop, Resize
)

def get_inference_transforms(CFG):
    return Compose([
            RandomCrop(512,512),
            Resize(CFG['img_size'], CFG['img_size']),
#            Transpose(p=0.5),
#            HorizontalFlip(p=0.5),
#            VerticalFlip(p=0.5),
#             HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)


In [ ]:
import glob
from tqdm import tqdm
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all
if __name__ == '__main__':
     # for training only, need nightly build pytorch

    seed_everything(CFG['seed'])
    
#     folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train.label.values)
    tst_preds = []
        
    if 1:
        # we'll train fold 0 first


        test = pd.DataFrame()
        test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
        test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', transforms=get_inference_transforms(CFG), output_label=False)

        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size=CFG['valid_bs'],
            num_workers=CFG['num_workers'],
            shuffle=False,
            pin_memory=False,
        )

        device = torch.device(CFG['device'])
#         if CFG['model_arch'] == 'resnext50_32x4d':
#             model = CustomResNext().to(device)
#         if 'efficientnet' in CFG['model_arch']:
#             model = CassvaImgClassifier(CFG['model_arch'], train.label.nunique()).to(device)

        #for epoch in range(CFG['epochs']-3):
        for (i,path) in enumerate(glob.glob('../input/vit5fold/*.pth')):    
            model.load_state_dict(torch.load(path)['model'])
            
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    #val_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader, device)]
                    tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]

        #val_preds = np.mean(val_preds, axis=0)         
        #print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_.label.values, val_preds)))
       #print('fold {} validation accuracy = {:.5f}'.format(fold, (valid_.label.values==np.argmax(val_preds, axis=1)).mean()))
        del model
        torch.cuda.empty_cache()

In [ ]:
tst_preds = np.mean(tst_preds, axis=0) 


In [ ]:
test_pred = pd.DataFrame(tst_preds)
test_pred['image_id'] = test.image_id

In [ ]:
test_pred = test_pred.set_index('image_id')

In [ ]:
test_pred.to_csv('vit_predictions.csv')

# Public Resnext

In [ ]:
# # ====================================================
# # Directory settings
# # ====================================================
# import os

# OUTPUT_DIR = './'
# MODEL_DIR = '../input/cassava-resnext50-32x4d-weights/'
# if not os.path.exists(OUTPUT_DIR):
#     os.makedirs(OUTPUT_DIR)
    
# TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
# TEST_PATH = '../input/cassava-leaf-disease-classification/test_images'

In [ ]:
# # ====================================================
# # CFG
# # ====================================================
# class CFG:
#     debug=False
#     num_workers=8
#     model_name='resnext50_32x4d'
#     size=512
#     batch_size=32
#     seed=2020
#     target_size=5
#     target_col='label'
#     n_fold=5
#     trn_fold=[0, 1, 2, 3, 4]
#     inference=True

In [ ]:
# # ====================================================
# # Library
# # ====================================================
# import sys
# sys.path.append('../input/pytorch-image-models/pytorch-image-models-master')

# import os
# import math
# import time
# import random
# import shutil
# from pathlib import Path
# from contextlib import contextmanager
# from collections import defaultdict, Counter

# import scipy as sp
# import numpy as np
# import pandas as pd

# from sklearn import preprocessing
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import StratifiedKFold

# from tqdm.auto import tqdm
# from functools import partial

# import cv2
# from PIL import Image

# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.optim import Adam, SGD
# import torchvision.models as models
# from torch.nn.parameter import Parameter
# from torch.utils.data import DataLoader, Dataset
# from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau

# import albumentations as A
# from albumentations.pytorch import ToTensorV2

# import timm

# import warnings 
# warnings.filterwarnings('ignore')

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# # ====================================================
# # Utils
# # ====================================================
# def get_score(y_true, y_pred):
#     return accuracy_score(y_true, y_pred)


# @contextmanager
# def timer(name):
#     t0 = time.time()
#     LOGGER.info(f'[{name}] start')
#     yield
#     LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s.')


# def init_logger(log_file=OUTPUT_DIR+'inference.log'):
#     from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
#     logger = getLogger(__name__)
#     logger.setLevel(INFO)
#     handler1 = StreamHandler()
#     handler1.setFormatter(Formatter("%(message)s"))
#     handler2 = FileHandler(filename=log_file)
#     handler2.setFormatter(Formatter("%(message)s"))
#     logger.addHandler(handler1)
#     logger.addHandler(handler2)
#     return logger

# #LOGGER = init_logger()

# def seed_torch(seed=42):
#     random.seed(seed)
#     os.environ['PYTHONHASHSEED'] = str(seed)
#     np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.backends.cudnn.deterministic = True

# seed_torch(seed=CFG.seed)

In [ ]:
# test = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
# test.head()

In [ ]:
# # ====================================================
# # Dataset
# # ====================================================
# class TestDataset(Dataset):
#     def __init__(self, df, transform=None):
#         self.df = df
#         self.file_names = df['image_id'].values
#         self.transform = transform
        
#     def __len__(self):
#         return len(self.df)

#     def __getitem__(self, idx):
#         file_name = self.file_names[idx]
#         file_path = f'{TEST_PATH}/{file_name}'
#         image = cv2.imread(file_path)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#         if self.transform:
#             augmented = self.transform(image=image)
#             image = augmented['image']
#         return image

In [ ]:
# # ====================================================
# # Transforms
# # ====================================================
# def get_transforms(*, data):
#     if data == 'valid':
#         return A.Compose([
#             A.Resize(CFG.size, CFG.size),
#             A.Normalize(
#                 mean=[0.485, 0.456, 0.406],
#                 std=[0.229, 0.224, 0.225],
#             ),
#             ToTensorV2(),
#         ])

In [ ]:
# # ====================================================
# # MODEL
# # ====================================================
# class CustomResNext(nn.Module):
#     def __init__(self, model_name='resnext50_32x4d', pretrained=False):
#         super().__init__()
#         self.model = timm.create_model(model_name, pretrained=pretrained)
#         n_features = self.model.fc.in_features
#         self.model.fc = nn.Linear(n_features, CFG.target_size)

#     def forward(self, x):
#         x = self.model(x)
#         return x

In [ ]:
# # ====================================================
# # Helper functions
# # ====================================================
# def load_state(model_path):
#     model = CustomResNext(CFG.model_name, pretrained=False)
#     try:  # single GPU model_file
#         model.load_state_dict(torch.load(model_path)['model'], strict=True)
#         state_dict = torch.load(model_path)['model']
#     except:  # multi GPU model_file
#         state_dict = torch.load(model_path)['model']
#         state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}

#     return state_dict


# def inference(model, states, test_loader, device):
#     model.to(device)
#     tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
#     probs = []
#     for i, (images) in tk0:
#         images = images.to(device)
#         avg_preds = []
#         for state in states:
#             model.load_state_dict(state)
#             model.eval()
#             with torch.no_grad():
#                 y_preds = model(images)
#             avg_preds.append(y_preds.softmax(1).to('cpu').numpy())
#         avg_preds = np.mean(avg_preds, axis=0)
#         probs.append(avg_preds)
#     probs = np.concatenate(probs)
#     return probs

In [ ]:
# # ====================================================
# # inference
# # ====================================================
# model = CustomResNext(CFG.model_name, pretrained=False)
# states = [load_state(MODEL_DIR+f'{CFG.model_name}_fold{fold}.pth') for fold in CFG.trn_fold]
# test_dataset = TestDataset(test, transform=get_transforms(data='valid'))
# test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, 
#                          num_workers=CFG.num_workers, pin_memory=True)
# resNext50_pred = inference(model, states, test_loader, device)
# # # submission
# # test['label'] = predictions.argmax(1)
# # test[['image_id', 'label']].to_csv(OUTPUT_DIR+'submission.csv', index=False)
# # test.head()

In [ ]:
# resNext50_sub = pd.DataFrame(resNext50_pred)
# resNext50_sub['image_id'] = test.image_id
# resNext50_sub = resNext50_sub.set_index('image_id')
# resNext50_sub = resNext50_sub.sort_index()
# resNext50_sub

In [ ]:
# resNext50_sub.to_csv('public_resnext_predictions.csv')

In [ ]:
pred1 = pd.read_csv('./predictions.csv',index_col=0).sort_index()
# pred2 = pd.read_csv('./public_resnext_predictions.csv',index_col=0).sort_index()
pred3 = pd.read_csv('./vit_predictions.csv',index_col=0).sort_index()
pred1 = pred1.div(pred1.sum(axis=1),axis=0)
# pred2 = pred2.div(pred2.sum(axis=1),axis=0)
pred3 = pred3.div(pred3.sum(axis=1),axis=0)

In [ ]:
pred1

In [ ]:
# pred2

In [ ]:
pred3

In [ ]:
submission = 0.8*pred1 + 0.2*pred3

In [ ]:
submission

In [ ]:
submission['label'] = submission.idxmax(axis=1)
submission = submission.reset_index()
submission

In [ ]:
submission[['image_id','label']].to_csv('submission.csv',index=False)